In [23]:
import yaml
import json
import pandas as pd
from pathlib import Path
from functools import reduce

SYMBOLS = list("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")

from pandas.io.formats import excel
excel.ExcelFormatter.header_style = None

In [24]:
path = Path("./data") / "zenit" / "zenit_i"
df = pd.read_excel(path / "dati_originali.xlsx")

In [25]:
unique_names = df["from"].sort_values().unique().tolist()
map_names = { k:v for k,v in zip(unique_names, SYMBOLS[:len(unique_names)]) }

(
    pd.Series(map_names)
        .to_frame()
        .reset_index()
        .rename(columns={"index": "soggetto", 0: "lettera"})
        .to_excel(path / "associazione.xlsx", index=False)
)

In [26]:
for ctx in ("lud", "stu"):
    
    df_yaml =  df.loc[:, ["from", f"to_pos_{ctx}", f"to_neg_{ctx}"]]
    df_yaml_converted = df_yaml.replace(map_names, regex=True).fillna("-")

    for type in ("pos", "neg"):
        with open(path / f"{ctx}_{type}.yaml", "w") as fout:
            data = df_yaml_converted.set_index("from").sort_index()[f"to_{type}_{ctx}"].str.replace(" ", "")
            data = [{k:v} for k, v in data.items()]
            y = yaml.dump(data).replace("'-'", "")
            fout.write(y)